# Data storage and project layout

- The initial dataset should be stored as `mol2` or `mol2.gz` files in a directory. For example:

In [8]:
!ls -lh 80698/mol2/

total 2343768
-rw-r--r--  1 sebastian  staff   572M May  6 21:20 1.mol2
-rw-r--r--  1 sebastian  staff   572M May  6 21:22 2.mol2


- for a new project, it's best to create a new directory, for instance, we create a new subdirectory in 80698:

In [2]:
!mkdir 80698/proj-1/

Using the `count_mol2.py` script, we can count the number of structures in each mol2 file in the input directory:

In [9]:
!python ../../../tools/count_mol2.py -i 80698/mol2

1.mol2 : 135966
2.mol2 : 136144
Total : 272110


Optionally, you may have files with data about the molecules, for instance:

In [ ]:
!ls -lh 80698/csv/

# General Blacklist & Whitelist filtering

- `mol2_to_id.py` generates a list of molecule IDs from MOL2 files
- `id_to_mol2.py` filters mol2 files by IDs and creates new mol2 files. Via whitelisting, molecules that match those IDs are written. Via blacklisting, all molecules but the molecules that are in the list are written

In [11]:
!python ../../../tools/mol2_to_id.py -i 80698/mol2 -o 80698/proj-1/all-mol2ids.txt

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [12]:
!head 80698/proj-1/all-mol2ids.txt

ZINC57271411
ZINC50764925
ZINC65255333
ZINC06394508
ZINC65292537
ZINC65375610
ZINC31820077
ZINC65395084
ZINC00205726
ZINC01458151


- Say we are interested in a subset of molecules only. Consider this example: 1st we create a list of IDs:

In [7]:
%%writefile ./80698/proj-1/selected-example-mol2ids.txt
ZINC57271411
ZINC06394508
ZINC65292537
ZINC31820077
ZINC01458151

Overwriting ./80698/proj-1/selected-example-mol2ids.txt


### Whitelisting

In [23]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/selected-example-mol2ids \
--id_file ./80698/proj-1/selected-example-mol2ids.txt \
--whitelist True

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


Now, the output directory, `80698/proj-1/selected-example-mol2ids`, should contain only mol2 files with the selected IDs:

In [24]:
!ls 80698/proj-1/selected-example-mol2ids

1.mol2 2.mol2


In [25]:
!grep 'ZINC' 80698/proj-1/selected-example-mol2ids/1.mol2

ZINC57271411
ZINC06394508
ZINC65292537
ZINC31820077
ZINC01458151


In [28]:
!grep -C 1 'ZINC' 80698/proj-1/selected-example-mol2ids/1.mol2 

@<TRIPOS>MOLECULE
ZINC57271411
   50    52     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC06394508
   49    52     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC65292537
   47    50     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC31820077
   48    50     0     0     0
--
--
@<TRIPOS>MOLECULE
ZINC01458151
   25    25     0     0     0


### Blacklisting

Similar to the previous approach, using a whitelist filter, we can do blacklist filtering, which means that all molecules are selected but the ones contained in the ID files. Set whitelist to False.

In [46]:
!python ../../../tools/id_to_mol2.py \
--input 80698/mol2 \
--output 80698/proj-1/nonselected-example-mol2ids \
--id_file ./80698/proj-1/selected-example-mol2ids.txt \
--whitelist false

Processing 80698/mol2/1.mol2
Processing 80698/mol2/2.mol2
Finished


In [47]:
!python ../../../tools/count_mol2.py -i 80698/proj-1/nonselected-example-mol2ids/

1.mol2 : 135961
2.mol2 : 136144
Total : 272105


# First Filtering Step -- Filtering via Features from Data Tables

- `id_from_table.py` -- this tools selects molecules IDs by user-specified filtering criteria based on the features stored in the columns of a data table
- the generic `mol2_from_id.py` script can then be used to create new mol2 files that either contain or don't contain molecules with certain IDs.